In [1]:
import networkx as nx
twitter: nx.DiGraph = nx.read_edgelist("./ego-Twitter/twitter_combined.txt.gz", create_using=nx.DiGraph)
twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()

(81306, 1768149, True)

In [2]:
#サンプリングしたグラフからコミュニティ検出（louvain法）
import networkx as nx

twitter: nx.DiGraph = nx.read_edgelist("sampled_twitter_graph.txt", create_using=nx.DiGraph)

twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()

twitter_comms = nx.community.louvain_communities(twitter)

print(f"検出されたコミュニティ数: {len(twitter_comms)}")

検出されたコミュニティ数: 51


In [ ]:
import os
import json # JSONライブラリを使用
import numpy as np
import shutil
# (networkx as nx もインポートされている前提)

# --- 1. 保存用フォルダの作成 ---
output_folder = "adj_json_communities_txt" # 保存先のフォルダ名
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"\n保存用フォルダ '{output_folder}' を作成しました。")
else:
    print(f"\n保存用フォルダ '{output_folder}' は既に存在します。")


# --- 2. 各コミュニティを指定の形式で .txt ファイルに保存 ---
print("各コミュニティを隣接行列の辞書形式で、txtファイルに保存しています...")

# twitter_comms が定義されている前提でループを開始
for i, community_nodes in enumerate(twitter_comms):
    if len(community_nodes) < 2:
        continue
    
    # Step 2-1: サブグラフを生成
    subgraph = twitter.subgraph(community_nodes)
    
    # Step 2-2: サブグラフから隣接行列を生成
    node_list = list(subgraph.nodes())
    adj_matrix_np = nx.to_numpy_array(subgraph, nodelist=node_list, dtype=int)
    
    # Step 2-3: NumPy配列をご希望の辞書形式に変換
    adjacency_dict = {}
    for row_idx, source_node in enumerate(node_list):
        row_dict = {}
        for col_idx, target_node in enumerate(node_list):
            row_dict[target_node] = int(adj_matrix_np[row_idx, col_idx])
        
        adjacency_dict[source_node] = row_dict
        
    # Step 2-4: ファイルパスを定義 (★拡張子を .txt に)
    file_path = os.path.join(output_folder, f"community_{i}.txt")
    
    # Step 2-5: TXTファイルとして書き出す (★json.dumpを使用)
    with open(file_path, 'w', encoding='utf-8') as f:
        # json.dumpで書き出すと、ご提示の例の形式と一致します
        # indent=2 を指定すると、例のように整形されて見やすくなります
        json.dump(adjacency_dict, f, indent=2) 

print(f"\n処理完了: {len(twitter_comms)}個のコミュニティを '{output_folder}' フォルダにTXTファイルとして保存しました。")


保存用フォルダ 'adj_json_communities_txt' を作成しました。
各コミュニティを隣接行列の辞書形式で、txtファイルに保存しています...

処理完了: 51個のコミュニティを 'adj_json_communities_txt' フォルダにTXTファイルとして保存しました。


In [ ]:
import os
import json # .txtファイルの中身がJSON形式なのでjsonライブラリを使います
import networkx as nx
import numpy as np 

# TXTファイルが保存されているフォルダ名
folder_name = "directed_communities_txt" 

# フォルダの存在確認
if not os.path.exists(folder_name):
    print(f"エラー: フォルダ '{folder_name}' が見つかりません。")
    print("先にファイル保存コードを実行してください。")
    exit()

print(f"フォルダ '{folder_name}' 内のファイルを確認します...\n")

try:
    txt_files = [f for f in os.listdir(folder_name) if f.endswith('.txt')]
    if not txt_files:
        print("エラー: フォルダ内にTXTファイルが見つかりません。")
        exit()
except Exception as e:
    print(f"エラー: フォルダの読み込み中に問題が発生しました。 {e}")
    exit()

# 有向グラフコミュニティの統計情報を格納するリスト
directed_communities_stats = [] 

# --- 1. 全てのファイルを有向グラフか判定し、係数を計算 ---
for filename in txt_files:
    file_path = os.path.join(folder_name, filename)
    
    try:
        # ご指定の通り、中身がJSONの.txtファイルを読み込みます
        with open(file_path, 'r', encoding='utf-8') as f:
            adj_dict = json.load(f)

        # ノード数が3未満（三角形が作れない）場合はスキップ
        if len(adj_dict) < 3:
            continue

        # --- 有向グラフ（非対称）であるか検証 ---
        nodes = list(adj_dict.keys())
        is_asymmetric = False # 非対称（有向グラフ）か
        asymmetry_proof = None # 非対称の証拠

        for i in range(len(nodes)):
            for j in range(i + 1, len(nodes)):
                node_a = nodes[i]
                node_b = nodes[j]
                
                # A->B と B->A が異なるかチェック
                if adj_dict[node_a][node_b] != adj_dict[node_b][node_a]:
                    is_asymmetric = True
                    asymmetry_proof = (node_a, node_b)
                    break
            if is_asymmetric:
                break
        
        # --- 非対称（有向グラフ）の場合のみ、リストに追加 ---
        if is_asymmetric:
            # グラフオブジェクトを復元
            G = nx.from_dict_of_dicts(adj_dict, create_using=nx.DiGraph)
            
            # クラスター係数を計算
            avg_cluster = nx.average_clustering(G)
            global_cluster = nx.transitivity(G)
            
            directed_communities_stats.append({
                "filename": filename,
                "nodes": G.number_of_nodes(),
                "edges": G.number_of_edges(),
                "avg_cluster": avg_cluster,
                "global_cluster": global_cluster,
                "proof": asymmetry_proof, 
                "adj_dict": adj_dict 
            })

    except Exception as e:
        print(f"エラー: ファイル '{filename}' の処理中に問題が発生しました。 {e}")
        continue # 問題のファイルはスキップ

# --- 2. ★★★ エラー修正箇所 ★★★ ---
# 分析対象のリストが空かどうかをチェック
if not directed_communities_stats:
    # リストが空（0件）の場合
    print("\n--- 分析結果 ---")
    print("分析対象となるコミュニティが見つかりませんでした。")
    print("（ノード数が3以上で、かつ非対称な関係を持つ有向グラフが0個でした）")

else:
    # リストに1件以上ある場合のみ、ソートと分析を実行
    # --- 3. 平均クラスター係数でソート ---
    sorted_communities = sorted(directed_communities_stats, key=lambda x: x["avg_cluster"])

    # --- 4. 3つのコミュニティを特定 ---
    lowest_comm = sorted_communities[0]
    highest_comm = sorted_communities[-1]

    all_avg_clusters = [c["avg_cluster"] for c in sorted_communities]
    median_value = np.median(all_avg_clusters)
    median_comm = min(sorted_communities, key=lambda x: abs(x["avg_cluster"] - median_value))

    # --- 5. 結果の詳細表示 ---
    print(f"--- 分析結果 ({len(directed_communities_stats)}個の有向グラフコミュニティ中) ---")

    def print_community_details(title, community_info):
        """コミュニティの詳細情報を表示する関数"""
        print(f"\n◆ {title}")
        print(f"  ファイル名: {community_info['filename']}")
        print(f"  ノード数  : {community_info['nodes']}")
        print(f"  エッジ数  : {community_info['edges']}")
        
        print("\n  --- クラスター係数 ---")
        print(f"  平均クラスター係数   : {community_info['avg_cluster']:.6f}")
        print(f"  グローバルクラスター係数: {community_info['global_cluster']:.6f}")
        
        print("\n  --- 有向グラフの検証 ---")
        proof_a, proof_b = community_info['proof']
        val_ab = community_info['adj_dict'][proof_a][proof_b]
        val_ba = community_info['adj_dict'][proof_b][proof_a]
        
        print(f"  ✅ 隣接行列は非対称です。")
        print(f"     証拠: ノード {proof_a} -> {proof_b} は [{val_ab}] ですが、")
        print(f"           ノード {proof_b} -> {proof_a} は [{val_ba}] です。")

    print_community_details("平均クラスター係数が【最も低い】コミュニティ", lowest_comm)
    print_community_details("平均クラスター係数が【中央値に最も近い】コミュニティ", median_comm)
    print_community_details("平均クラスター係数が【最も高い】コミュニティ", highest_comm)

フォルダ 'directed_communities_txt' 内のファイルを確認します...

エラー: ファイル 'community_1.txt' の処理中に問題が発生しました。 to_dict_of_dicts() got an unexpected keyword argument 'create_using'
エラー: ファイル 'community_0.txt' の処理中に問題が発生しました。 to_dict_of_dicts() got an unexpected keyword argument 'create_using'
エラー: ファイル 'community_2.txt' の処理中に問題が発生しました。 to_dict_of_dicts() got an unexpected keyword argument 'create_using'
エラー: ファイル 'community_3.txt' の処理中に問題が発生しました。 to_dict_of_dicts() got an unexpected keyword argument 'create_using'
エラー: ファイル 'community_7.txt' の処理中に問題が発生しました。 to_dict_of_dicts() got an unexpected keyword argument 'create_using'
エラー: ファイル 'community_6.txt' の処理中に問題が発生しました。 to_dict_of_dicts() got an unexpected keyword argument 'create_using'
エラー: ファイル 'community_4.txt' の処理中に問題が発生しました。 to_dict_of_dicts() got an unexpected keyword argument 'create_using'
エラー: ファイル 'community_5.txt' の処理中に問題が発生しました。 to_dict_of_dicts() got an unexpected keyword argument 'create_using'
エラー: ファイル 'community_13.txt' の処理中に問題が発生しました。 to

In [1]:
import os
import json # JSONファイルを読み込むため
import networkx as nx

# --- 設定 ---
# 1. 保存コードで指定したフォルダ名
folder_name = "adj_json_communities_txt" 

# 2. ★分析したいファイル名を指定★
file_to_analyze = "community_0.txt"
# ----------------

# ファイルのフルパスを作成
file_path = os.path.join(folder_name, file_to_analyze)

try:
    # --- 1. JSON形式のTXTファイルを読み込む ---
    print(f"ファイル '{file_path}' を読み込んでいます...")
    with open(file_path, 'r', encoding='utf-8') as f:
        # ご提示のコード(json.dump)で保存したため、json.loadで読み込みます
        adj_dict = json.load(f)

    # --- 2. グラフオブジェクトを復元 ---
    print("nx.from_dict_of_dicts でグラフを復元しています...")
    # この adj_dict は「辞書の辞書」形式なので、この関数で正しく読み込めます
    G = nx.from_dict_of_dicts(adj_dict, create_using=nx.DiGraph)

    # ノード数が3未満かチェック
    if G.number_of_nodes() < 3:
        print(f"ノード数: {G.number_of_nodes()}")
        print("エラー: ノード数が3未満のため、クラスター係数を計算できません。")
    else:
        # --- 3. 平均クラスター係数を計算 ---
        print("平均クラスター係数を計算中...")
        avg_cluster = nx.average_clustering(G)
        
        # --- 4. 結果を表示 ---
        print("\n--- 計算結果 ---")
        print(f"ファイル名: {file_to_analyze}")
        print(f"ノード数: {G.number_of_nodes()}")
        print(f"エッジ数: {G.number_of_edges()}")
        print(f"平均クラスター係数: {avg_cluster:.6f}")

except FileNotFoundError:
    print(f"エラー: ファイル '{file_path}' が見つかりません。")
except json.JSONDecodeError:
    print(f"エラー: ファイル '{file_path}' は正しいJSON形式ではありません。")
except Exception as e:
    # 'int' object is not iterable などのエラーはここで捕捉されます
    print(f"エラー: 処理中に予期せぬ問題が発生しました。 {e}")

ファイル 'adj_json_communities_txt/community_0.txt' を読み込んでいます...
nx.from_dict_of_dicts でグラフを復元しています...
エラー: 処理中に予期せぬ問題が発生しました。 'int' object is not iterable


In [ ]:
import os
import json
import networkx as nx
import numpy as np

# TXTファイルが保存されているフォルダ名
folder_name = "directed_communities_txt" 

# フォルダの存在確認
if not os.path.exists(folder_name):
    print(f"エラー: フォルダ '{folder_name}' が見つかりません。")
    print("先にファイル保存コードを実行してください。")
    exit()

print(f"フォルダ '{folder_name}' 内のファイルを分析します...\n")

try:
    # 拡張子が.txtのファイルを探します
    txt_files = [f for f in os.listdir(folder_name) if f.endswith('.txt')]
    if not txt_files:
        print("エラー: フォルダ内にTXTファイルが見つかりません。")
        exit()
except Exception as e:
    print(f"エラー: フォルダの読み込み中に問題が発生しました。 {e}")
    exit()

# 各コミュニティの統計情報を格納するリスト
community_stats = []

# --- 1. 全てのファイルを読み込み、平均クラスター係数を計算 ---
for filename in txt_files:
    file_path = os.path.join(folder_name, filename)
    
    try:
        # JSON形式の.txtファイルを読み込みます
        with open(file_path, 'r', encoding='utf-8') as f:
            adj_dict = json.load(f)

        # ノード数が3未満（三角形が作れない）場合はスキップ
        if len(adj_dict) < 3:
            continue

        # 辞書から有向グラフを復元
        # クラスター係数の計算では有向グラフ(DiGraph)として扱うのが適切です
        G = nx.read_edgelist(adj_dict, create_using=nx.DiGraph)

        # 平均クラスター係数を計算
        avg_cluster = nx.average_clustering(G)
        
        # 結果をリストに追加
        community_stats.append({
            "filename": filename,
            "nodes": G.number_of_nodes(),
            "avg_cluster": avg_cluster,
        })

    except Exception as e:
        print(f"エラー: ファイル '{filename}' の処理中に問題が発生しました。 {e}")
        continue # 問題のファイルはスキップ

# --- 2. リストが空でないか確認 ---
if not community_stats:
    print("\n--- 分析結果 ---")
    print("分析対象となるコミュニティが見つかりませんでした。")
    print("（ノード数が3以上のコミュニティが0個でした）")
    exit()

# --- 3. 平均クラスター係数でソート ---
sorted_communities = sorted(community_stats, key=lambda x: x["avg_cluster"])

# --- 4. 3つのコミュニティを特定 ---
# 最も低いコミュニティ
lowest_comm = sorted_communities[0]

# 最も高いコミュニティ
highest_comm = sorted_communities[-1]

# 中央値に最も近いコミュニティ
all_avg_clusters = [c["avg_cluster"] for c in sorted_communities]
median_value = np.median(all_avg_clusters)
median_comm = min(sorted_communities, key=lambda x: abs(x["avg_cluster"] - median_value))

# --- 5. 結果の詳細表示 ---
print(f"--- 分析結果 ({len(community_stats)}個のコミュニティ中) ---")

def print_community_details(title, community_info):
    """コミュニティの詳細情報を表示する関数"""
    print(f"\n◆ {title}")
    print(f"  ファイル名: {community_info['filename']}")
    print(f"  ノード数  : {community_info['nodes']}")
    print(f"  平均クラスター係数: {community_info['avg_cluster']:.6f}")

print_community_details("平均クラスター係数が【最も低い】コミュニティ", lowest_comm)
print_community_details("平均クラスター係数が【中央値に最も近い】コミュニティ", median_comm)
print_community_details("平均クラスター係数が【最も高い】コミュニティ", highest_comm)

フォルダ 'directed_communities_txt' 内のファイルを分析します...

エラー: ファイル 'community_1.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_0.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_2.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_3.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_7.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_6.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_4.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_5.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_13.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_12.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_10.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_38.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_39.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_11.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_29.txt' の処理中に問題が発生しました。 division by zero
エラー: ファイル 'community_15.txt' 

IndexError: list index out of range

: 